In [17]:
import pandas as pd
import numpy as np

In [14]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data

--2022-03-10 12:40:55--  https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49082 (48K) [application/x-httpd-php]
Saving to: ‘housing.data’

housing.data        100%[===================>]  47.93K  --.-KB/s    in 0.1s    

2022-03-10 12:40:56 (333 KB/s) - ‘housing.data’ saved [49082/49082]



In [18]:
#Import dataset and add headers
dataset=pd.read_csv("housing.data",delim_whitespace=True,
                    names=["crim","zn","indus",
                           "chas","nox","rm",
                           "age","dis","rad",
                           "tax","ptratio","black",
                           "lstat","medv"])

In [19]:
dataset.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [29]:
dataset.size

7084

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [32]:
original_df, shadow_df = train_test_split(dataset, test_size=0.5, random_state=101)

In [33]:
original_df.size

3542

In [35]:
X = original_df.drop(columns=['medv'])
Y = original_df['medv']

x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.25, random_state=101)

In [36]:
from sklearn.ensemble import ExtraTreesRegressor

In [37]:
def target_model_fn():
  model = ExtraTreesRegressor(n_estimators = 500)
  model.fit(x_train,y_train)
  train_pred = model.predict(x_train)
  valid_pred = model.predict(x_valid)

  print(f"Score = {model.score(x_train,y_train)}\tTrain MSE = {mean_squared_error(train_pred, y_train)}")
  print(f"Val Score = {model.score(x_valid,y_valid)}\t Val MSE = {mean_squared_error(valid_pred, y_valid)}")
  return model

**Now jumping to MIA !!!**

In [26]:
!pip install mia

In [64]:
Xshadow_df = shadow_df.drop(columns=['medv'])
Yshadow_df = shadow_df['medv']
attacker_X_train, attacker_X_test, attacker_y_train, attacker_y_test = train_test_split(Xshadow_df, Yshadow_df, test_size=0.1)

In [65]:
from mia.estimators import ShadowModelBundle
SHADOW_DATASET_SIZE=126
NUM_MODELS=5

smb = ShadowModelBundle(target_model_fn, shadow_dataset_size=SHADOW_DATASET_SIZE, num_models=NUM_MODELS)
X_shadow, y_shadow = smb.fit_transform(attacker_X_train, attacker_y_train)


ValueError: ignored